# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [2]:
from pyspark.sql import SparkSession


# 1) import any other libraries you might need
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import desc
from pyspark.sql.functions import sum as Fsum

# 2) instantiate a Spark session 
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

# 3) read in the data set located at the path "data/sparkify_log_small.json"
path = 'data/sparkify_log_small.json'
user_log = spark.read.json(path)

# 4) write code to answer the quiz questions 

In [10]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [16]:
user_log.take(5)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046'),
 Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000'),
 Row(artist='Cobra Starship Featuring Leighton Meester', auth='Logged In', firstNa

# Question 1

Which page did user id "" (empty string) NOT visit?

In [13]:
# TODO: write your code to answer question 1
user_log.select(["page"]).where(user_log.userId == "").dropDuplicates().collect()

[Row(auth='Logged Out'), Row(auth='Guest')]

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [17]:
# TODO: use this space to explore the behavior of the user with an empty string
user_log.select(["page", "auth"]).where(user_log.userId == "").dropDuplicates().collect()

462

# Question 3

How many female users do we have in the data set?

In [3]:
# TODO: write your code to answer question 3
user_log.select(["userId"]).where(user_log.gender == "F").dropDuplicates().count()

# Question 4

How many songs were played from the most played artist?

In [6]:
# TODO: write your code to answer question 4
user_log.filter(user_log.page == "NextSong").select("artist").groupby("artist").count().sort("count", ascending = False).show(10)

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|        Jack Johnson|   36|
|                Muse|   36|
+--------------------+-----+
only showing top 10 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [108]:
# TODO: write your code to answer question 5
from pyspark.sql import Window

windowval = Window.partitionBy("userId").orderBy("ts").rangeBetween(Window.unboundedPreceding, 0)

In [109]:
flag_songplay_event = udf(lambda x: 1 if x == "NextSong" else 0, IntegerType())

In [110]:
play_log = user_log.withColumn("playsong", flag_songplay_event("page"))

In [111]:
play_log.withColumn("cumsum", Fsum("playsong").over(windowval)).printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- playsong: integer (nullable = true)
 |-- cumsum: long (nullable = true)



In [112]:
play_log.select('ts', 'page', 'userId', 'cumsum' , 'sessionId' ).filter(" userId == '2294' ").take(100)

AnalysisException: "cannot resolve '`cumsum`' given input columns: [method, lastName, gender, artist, userAgent, song, registration, firstName, itemInSession, status, ts, page, sessionId, location, playsong, level, auth, length, userId];;\n'Project [ts#3644L, page#3639, userId#3646, 'cumsum, sessionId#3641L]\n+- Project [artist#3629, auth#3630, firstName#3631, gender#3632, itemInSession#3633L, lastName#3634, length#3635, level#3636, location#3637, method#3638, page#3639, registration#3640L, sessionId#3641L, song#3642, status#3643L, ts#3644L, userAgent#3645, userId#3646, <lambda>(page#3639) AS playsong#8191]\n   +- Relation[artist#3629,auth#3630,firstName#3631,gender#3632,itemInSession#3633L,lastName#3634,length#3635,level#3636,location#3637,method#3638,page#3639,registration#3640L,sessionId#3641L,song#3642,status#3643L,ts#3644L,userAgent#3645,userId#3646] json\n"

In [114]:
play_log.filter(play_log.page == 'Home').select('page', 'userId', 'cumsum' , 'sessionId' )\
    .groupBy().avg('cumsum').collect()

AnalysisException: "cannot resolve '`cumsum`' given input columns: [method, lastName, gender, artist, userAgent, song, registration, firstName, itemInSession, status, ts, page, sessionId, location, playsong, level, auth, length, userId];;\n'Project [page#3639, userId#3646, 'cumsum, sessionId#3641L]\n+- Filter (page#3639 = Home)\n   +- Project [artist#3629, auth#3630, firstName#3631, gender#3632, itemInSession#3633L, lastName#3634, length#3635, level#3636, location#3637, method#3638, page#3639, registration#3640L, sessionId#3641L, song#3642, status#3643L, ts#3644L, userAgent#3645, userId#3646, <lambda>(page#3639) AS playsong#8191]\n      +- Relation[artist#3629,auth#3630,firstName#3631,gender#3632,itemInSession#3633L,lastName#3634,length#3635,level#3636,location#3637,method#3638,page#3639,registration#3640L,sessionId#3641L,song#3642,status#3643L,ts#3644L,userAgent#3645,userId#3646] json\n"

In [53]:
user_log.filter(user_log.page.isin({'NextSong', 'Home')).select('page', 'userId').collect()

[Row(page='NextSong', userId='1046'),
 Row(page='NextSong', userId='1000'),
 Row(page='NextSong', userId='2219'),
 Row(page='NextSong', userId='2373'),
 Row(page='Home', userId='1747'),
 Row(page='NextSong', userId='1162'),
 Row(page='NextSong', userId='1061'),
 Row(page='Home', userId='748'),
 Row(page='Home', userId='597'),
 Row(page='NextSong', userId='1806'),
 Row(page='NextSong', userId='748'),
 Row(page='NextSong', userId='1176'),
 Row(page='NextSong', userId='2164'),
 Row(page='NextSong', userId='2146'),
 Row(page='NextSong', userId='2219'),
 Row(page='Home', userId='1176'),
 Row(page='NextSong', userId='2904'),
 Row(page='NextSong', userId='597'),
 Row(page='NextSong', userId='226'),
 Row(page='NextSong', userId='1046'),
 Row(page='NextSong', userId='1162'),
 Row(page='NextSong', userId='2942'),
 Row(page='NextSong', userId='2779'),
 Row(page='NextSong', userId='1806'),
 Row(page='NextSong', userId='2887'),
 Row(page='NextSong', userId='1035'),
 Row(page='NextSong', userId='748

In [27]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

